In [1]:
import bz2
import json
import re
import random
import sys
import os
import bz2
import time
import pandas as pd
import numpy as np
from functools import partial

In [2]:
signi_quotes_file_names = [
    "./data/signi-quote-count-2015_1636244638891.json",
    "./data/signi-quote-count-2016_1636246832187.json",
    "./data/signi-quote-count-2017_1636249273913.json",
    "./data/signi-quote-count-2018_1636250518608.json",
    "./data/signi-quote-count-2019_1636251729971.json",
    "./data/signi-quote-count-2020_1636237785105.json"
]
# json.load(open(signi_quotes_file_names[0]))

In [3]:

by_year_dict = {}
for file_name in signi_quotes_file_names:
    combined_signi_dict = {}
    with open(file_name, 'r') as f:
        one_dict = json.load(f)
        for k in one_dict.keys():
            combined_signi_dict[k] = combined_signi_dict.get(k, 0) + one_dict[k]
    by_year_dict[file_name] = combined_signi_dict
    

filter quotes by year

In [15]:
with open('./data/top100_politicians_by_party.json', 'r') as f:
    data_top100 = json.load(f)

dem_list = pd.DataFrame(data_top100["dem"])
rep_list = pd.DataFrame(data_top100["rep"])
print(dem_list)

        item        itemLabel genderLabel          citizenshipLabel  \
0        Q76     Barack Obama        male  United States of America   
1      Q6294  Hillary Clinton      female  United States of America   
2    Q359442   Bernie Sanders        male  United States of America   
3    Q170581     Nancy Pelosi      female  United States of America   
4     Q11673     Andrew Cuomo        male  United States of America   
..       ...              ...         ...                       ...   
95   Q505598   Jerrold Nadler        male  United States of America   
96   Q312644      Eric Holder        male  United States of America   
97  Q6533414    Letitia James      female  United States of America   
98    Q19673          Al Gore        male  United States of America   
99  Q7595813    Stacey Abrams      female  United States of America   

   languageLabel                                      religionLabel  \
0        English  [Congregationalist polity, United Church of Ch...   
1    

In [94]:
poli_quotes_combined = {}
for qid in rep_list["item"]:
    poli_quotes_combined[qid] = {}
    poli_quotes_combined[qid]["quotations"] = []

for qid in dem_list["item"]:
    poli_quotes_combined[qid] = {}
    poli_quotes_combined[qid]["quotations"] = []

In [29]:
# poli_quotes_combined

In [39]:
"data/politician-quotes-2015_1636331534906.json".find("2")
"data/politician-quotes-2015_1636331534906.json"[23:27]

'2015'

In [97]:
def write_json_to_file(name, obj):
    # Use current timestamp to make the name of the file unique
    millis = round(time.time() * 1000)
    name = f'{name}.json'
    with open(name, 'wb') as f:
        output = json.dumps(obj)
        f.write(output.encode('utf-8'))
    return name

In [99]:
import sys
poli_quote_files = [
    "data/politician-quotes-2015_1636331534906.json",
    "data/politician-quotes-2016_1636332058163.json",
    "data/politician-quotes-2017_1636333168732.json",
    "data/politician-quotes-2018_1636334221167.json",
    "data/politician-quotes-2019_1636335010497.json",
    "data/politician-quotes-2020_1636330658142.json"
]



# both_parties = dem_list + rep_list
# for v in both_parties:
#     copy = dict(v)
#     copy['quotations'] = []
    
#     poli_quotes_combined[v['id']] = copy

for poli_quote_file_name in poli_quote_files:
    # For each year
    poli_quotes_combined = {}
    for qid in rep_list["item"]:
        poli_quotes_combined[qid] = {}
        poli_quotes_combined[qid]["quotations"] = []

    for qid in dem_list["item"]:
        poli_quotes_combined[qid] = {}
        poli_quotes_combined[qid]["quotations"] = []
    with open(poli_quote_file_name, 'r') as f:
        quotes = json.load(f)
        # print(quotes)
        
        for k in quotes.keys():
            # print(k)
            if k in poli_quotes_combined:
                poli_quotes_combined[k]['quotations'] += quotes[k]
                # print(poli_quotes_combined[k])
        write_json_to_file('data/combined_' + poli_quote_file_name[23:27], poli_quotes_combined)
        # sys.exit()


In [100]:
def get_good_qid_quote(name):
        
    with open(name, 'r') as f:
        poli_quotes_filtered = json.load(f)
    filtered_quotes = []

    weird_pattern = '[_@#+&;:\(\)\{\}\[\]\\/`]'
    json_pattern = '\{.*[a-zA-Z]+:\s[\'"`][a-zA-Z0-9]+[\'"`].*\}'
    url_pattern = 'https?'

    for k in poli_quotes_filtered.keys():
        elem = poli_quotes_filtered[k]
        
        new_arr = []
        for entry in elem['quotations']:
            text = entry['quotation']
            
            longest = max(entry['quotation'].split(), key=len)
            if (len(longest) > 50):
                filtered_quotes.append(entry)
                continue
            
            if re.search(url_pattern, text) is not None:
                filtered_quotes.append(entry)
                continue
            
            if re.search(json_pattern, text) is not None:
                filtered_quotes.append(entry)
                continue
                
            weird_num = len(re.findall(weird_pattern, text))
            total = len(text)
            weird_percent = weird_num / total
            if (weird_percent > 0.1):
                filtered_quotes.append(entry)
                continue
                
            new_arr.append(entry)
        elem['quotations'] = new_arr

    poli_quotes_concat = poli_quotes_filtered
    QUOTE_LENGTH = 5000

    for k in poli_quotes_concat.keys():
        elem = poli_quotes_concat[k]
        
        # Sort the quotes by length
        elem['quotations'].sort(key = lambda x: len(x['quotation']), reverse = True)
        
        concat = ''
        for quote in elem['quotations']:
            # Concatenate the quotes
            concat += ' ' + quote['quotation']
            
            # Trim if we are over QUOTE_LENGTH
            if (len(concat) >= QUOTE_LENGTH):
                concat = concat[0:QUOTE_LENGTH]
                break
        elem['quotations'] = concat
    write_json_to_file('data/yiren_'+name[14:18], poli_quotes_concat)
    return poli_quotes_concat
    

In [55]:
# "data/combined_2015_1638980597622.json".find("20")

14

In [101]:
for i in range(2015, 2021):
    get_good_qid_quote("data/combined_" + str(i) + ".json")
# get_good_qid_quote('data/combined_2015_1638980597622.json')
# get_good_qid_quote('data/combined_2016_1638980699759.json')
# get_good_qid_quote('data/combined_2017_1638980701297.json')
# get_good_qid_quote('data/combined_2018_1638980703329.json')
# get_good_qid_quote('data/combined_2019_1638980705776.json')
# get_good_qid_quote('data/combined_2020_1638980708411.json')



In [59]:
import bz2
import json
import pandas as pd
import csv

In [60]:
with open('./data/yiren_2015_1638981321807.json', 'r') as f:
    data = json.load(f)
with open('yiren_data2015.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["qid", "quote"])
    for qid, all_value in data.items():
        quote = all_value["quotations"]
        writer.writerow([qid, quote])

In [67]:
# ls =  ["./data/" + i for i in  ["yiren_2016_1638981329063.json", "yiren_2017_1638981338905.json", "yiren_2018_1638981351821.json", "yiren_2019_1638981366026.json", "yiren_2020_1638981381019.json"]]

In [102]:
ls = []
for i in range(2015, 2021):
    ls.append("./data/yiren_" + str(i) + ".json")

In [103]:
# './data/yiren_2016_1638981329063.json'.find("20")

In [104]:
def get_csv(name):
    with open(name, 'r') as f:
        data = json.load(f)
    with open('yiren_data' + name[13:17] + '.csv', 'w', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["qid", "quote"])
        for qid, all_value in data.items():
            quote = all_value["quotations"]
            writer.writerow([qid, quote])

In [105]:
[get_csv(name) for name in ls]

[None, None, None, None, None, None]

In [73]:
# liwc = pd.read_csv('liwc2015.csv')

In [74]:
liwc.sample()

,Source (A),Source (B),WC,WPS,Sixltr,Dic,Pronoun,I,We,Self,...,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP
107,Q4911497,As conveyed by Mayor de Blasio and Commission...,831,16.96,24.79,69.19,6.98,1.32,2.41,3.73,...,4.93,0.0,0.0,0.0,0.0,1.56,0.0,0.96,0.0,0.24


In [106]:
import helpers
with open('./data/top100_politicians_by_party.json', 'r') as f:
    data_top100 = json.load(f)

dem_df = pd.DataFrame(data_top100["dem"])
rep_df = pd.DataFrame(data_top100["rep"])
dem_df['party'] = "dem"
rep_df['party'] = "rep"
politician_wiki = pd.concat([dem_df, rep_df])

In [89]:
liwc = pd.read_csv('liwc2016.csv')
personality = helpers.predict_personality(liwc)
df1 = pd.concat([liwc, personality], axis=1)
df2 = df1.merge(politician_wiki, left_on='Source (A)', right_on='item', how = "left")
df2.to_csv("2016_for_r.csv",index=False)

In [107]:
def for_r(year):
    liwc = pd.read_csv('liwc'+str(year)+'.csv')
    personality = helpers.predict_personality(liwc)
    df1 = pd.concat([liwc, personality], axis=1)
    df2 = df1.merge(politician_wiki, left_on='Source (A)', right_on='item', how = "left")
    df2.to_csv(str(year) + "_for_r.csv",index=False)

In [108]:
for i in range(2015, 2021):
    for_r(i)